## 3.3

### 3.3.1

In [3]:
import torch

In [4]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

In [ ]:
# calculating attention scores $omega$

query = inputs[1]
attn_scores2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores2[i] = torch.dot(x_i, query)
print(attn_scores2)

#normalizing attentions scores
# In practice, it’s more common and advisable to use the softmax function for normalization.
attn_weights2_tmp = attn_scores2 / attn_scores2.sum()
print(attn_weights2_tmp, f"\n{attn_weights2_tmp.sum()}")

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656]) 
1.0000001192092896


In [ ]:
def softmax_naive(x: torch.Tensor):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

attn_weights2_naive = softmax_naive(attn_scores2)
print(attn_weights2_naive, f"\n{attn_weights2_naive.sum()}")

# since torch's implementation is more stable and is optimized for performance
attn_weights2 = torch.softmax(attn_scores2, dim=0)
print(attn_weights2, f"\n{attn_weights2.sum()}")

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]) 
1.0
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]) 
1.0


In [13]:
# calculating context vector z^(2)
query = inputs[1]
context_vect2 = torch.zeros(query.shape[0])
for i, x_i in enumerate(inputs):
    context_vect2 += attn_weights2[i] * x_i
print(context_vect2)

tensor([0.4419, 0.6515, 0.5683])
